In [16]:
#open pickle file with dataframe
import pickle
import pandas as pd
with open('../datafiles/filtered_data.pkl', 'rb') as f:
    df = pickle.load(f)
pd.set_option('display.max_columns', None)

C:\Users\case4\AppData\Local\Temp\ipykernel_9032\1571859427.py:5: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df = pickle.load(f)


In [18]:
df.head(100)

,Status/Disposal,Ear Tag,Ear Tag Location,Brand,Brand Location,Tattoo,Tattoo Location,Other ID,Other ID Location,Name,Registration Number,Breeder,Breeder Member Number,Owner,Owner Member Number,Date of Birth,Birth Method,Sire,Dam,Status,Type or Sex,Percent Purebred,Breed,Breed 1,Breed 2,Breed 3,Breed Comp 1,Breed Comp 2,Breed Comp 3,Horns,Color Markings,EPDType,EPD Reporting Period,CED,CED Acc,BW,BW Acc,WW,WW Acc,YW,YW Acc,MK,MK Acc,TM,Calving Ease Maternal EPD,Calving Ease Maternal Acc,Scrotal Circumference EPD,Scrotal Circumference Acc,Ribeye Area EPD,Ribeye Area Acc,Marbling EPD,Marbling Acc,Fat Thickness EPD,Fat Thickness Acc,Carcass Weight EPD,Carcass Weight Acc,Percent Retail Cuts EPD,Percent Retail Cuts Acc,Intramuscular Fat EPD,Intramuscular Fat Acc,Birth Weight,Adjusted Birth Weight,Birth Contemp Group,Birth contemporaries,Birth Weight Ratio,Birth Height,Calving Ease,Birth Code,Nurse Code,Twin code,Dam Teat Score,Dam Udder Score,Dam Gestation Length,Weaning Date,Weaning Weight Date,Weaning Weight,Adjusted Weaning Weight,Weaning Contemp Group,Weaning Contemporaries,Weaning Weight Ratio,Weaning WDA,Weaning Height,Weaning Frame Score,Weaning Mgmt Code,Weaning Pasture Code,Weaning Sheath/Namvel,Yearling Date,Yearling Weight Date,Yearling Weight,Adjusted Yearling Weight,Yearling Contemp Group,Yearling Contemporaries,Yearling Weight Ratio,Yearling WDA,Yearling ADG,Yearling ADG Ratio,Yearling Height,Yearling Frame Score,Yearling Mgmt Code,Yearling Pasture Code,Yearling Sheath/Navel,Growth,Year_Born,Age,Designation,Composite Score
0,0,NaN,NaN,C0271,RH,8075,LE,NaN,NaN,ARTESIAN CATTLE C0271188075,1A128345,ARTESIAN CATTLE & FARMING LLC,291.0,ARTESIAN CATTLE & FARMING LLC,291.0,2018-03-05,NS,AF31519,X93873,Active,C,NaN,NaN,AA,AN,XX,50.0,25.0,25.0,P,NaN,P,20240808.0,-0.5,0.10,0.4,0.08,47.0,0.06,83.0,0.01,24.0,0.04,46.999000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,6144-6271,40.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.0316,2018.0,6.0,Dam,3.850409
1,0,NaN,NaN,C0271,RH,9071,NaN,NaN,NaN,ARTESIAN CATTLE 9071,1A151658,ARTESIAN CATTLE & FARMING LLC,291.0,ARTESIAN CATTLE & FARMING LLC,291.0,2019-02-26,NS,AF36497,X94219,Active,C,NaN,NaN,AA,CH,XX,50.0,25.0,25.0,P,NaN,P,20240808.0,-3.8,0.12,2.3,0.08,46.0,0.05,84.0,0.01,26.0,0.04,49.418000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,15786-6,22.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.5187,2019.0,5.0,Dam,3.594708
2,0,NaN,NaN,C0271,RH,1027,NaN,NaN,NaN,ARTESIAN 1603Z OLIVIA 1027,1A217728,ARTESIAN CATTLE & FARMING LLC,291.0,ARTESIAN CATTLE & FARMING LLC,291.0,2021-03-16,NS,AF36497,X93901,Active,C,NaN,NaN,AA,AN,XX,50.0,25.0,25.0,P,NaN,P,20240808.0,-3.8,0.12,2.3,0.08,46.0,0.05,84.0,0.01,26.0,0.04,49.418000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,18216-3,66.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.5187,2021.0,3.0,Dam,3.594708
4,0,NaN,NaN,C0271,RH,1080,NaN,NaN,NaN,ARTESIAN CATTLE 1080,1A217781,ARTESIAN CATTLE & FARMING LLC,291.0,ARTESIAN CATTLE & FARMING LLC,291.0,2021-02-22,NS,AF42330,X209905,Active,C,NaN,NaN,AA,AR,NaN,50.0,50.0,NaN,P,NaN,P,20240808.0,-5.1,0.11,2.1,0.08,42.0,0.05,77.0,0.01,27.0,0.04,48.215500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,18217-1,13.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.7369,2021.0,3.0,Dam,3.363317
5,0,NaN,NaN,C0271,RH,1097,NaN,NaN,NaN,ARTESIAN 2163A AIDEN 1097,1A217798,ARTESIAN CATTLE & FARMING LLC,291.0,ARTESIAN CATTLE & FARMING LLC,291.0,2021-02-26,NS,AF42330,X209908,Active,C,NaN,NaN,AA,AR,NaN,50.0,50.0,NaN

In [13]:
df.loc[df['Categories'] == "High"]["CED"][1]

'15'